In [8]:
import random
import pygame
import math
import time


In [9]:
class Flower:
    def __init__(self):
        self.genes = [
            random.randint(1, 10),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 7)
        ]

In [10]:
def create_population(n):
    population = []
    for _ in range(n):
        population.append(Flower())
    return population

In [11]:
flowers = create_population(8)
for f in flowers:
    print(f.genes)

[6, 121, 53, 227, 240, 55, 92, 242, 216, 82, 6]
[7, 169, 67, 166, 118, 119, 93, 224, 134, 219, 0]
[9, 102, 244, 253, 236, 201, 16, 53, 180, 102, 2]
[2, 16, 139, 198, 103, 81, 146, 247, 166, 56, 5]
[7, 15, 232, 253, 167, 52, 0, 63, 63, 64, 2]
[5, 250, 111, 52, 114, 11, 55, 8, 222, 232, 5]
[4, 116, 254, 73, 132, 144, 199, 19, 177, 200, 5]
[7, 29, 114, 56, 118, 216, 113, 255, 133, 222, 5]


In [12]:
for f in flowers:
    f.fitness = 0.0
    f.hover_time = 0 

def render_flower(screen, flower, x, cursor):

    size, r_cen, g_cen, b_cen, \
    r_pet, g_pet, b_pet, \
    r_stm, g_stm, b_stm, \
    petal_count = flower.genes
    
    Y = 300 

    center_color = (r_cen, g_cen, b_cen)
    petal_color = (r_pet, g_pet, b_pet)
    stem_color = (r_stm, g_stm, b_stm)

    pygame.draw.line(screen, stem_color, (x, Y + 100), (x, Y), 4)

    if petal_count > 0:
        petal_radius = size + 15
        petal_size = 20
        for i in range(petal_count):
            angle = math.radians(i * (360 / petal_count))
            px = x + int(math.cos(angle) * petal_radius)
            py = Y - int(math.sin(angle) * petal_radius)
            pygame.draw.circle(screen, petal_color, (px, py), petal_size)

    pygame.draw.circle(screen, center_color, (x, Y), size)
    
    hit_box = pygame.Rect(x - 50, Y - 50, 100, 100)
    
    is_being_hovered = hit_box.collidepoint(cursor)
    
    if is_being_hovered:
        pygame.draw.rect(screen, (255, 255, 0), hit_box, 3)
    

    ui_font = pygame.font.Font(None, 24)
    score_display = ui_font.render(f"Fitness: {flwr.fitness:.1f}", True, (0, 0, 0))
    screen.blit(score_display, (x - 30, Y - 80))
    
    return hit_box, is_being_hovered


In [13]:
import random

def weighted_roulette_selection(flowers):
    fitness_values = [flower.fitness for flower in flowers]
    
    min_fitness = min(fitness_values)
    if min_fitness <= 0:
        adjusted_fitness = [f - min_fitness + 0.1 for f in fitness_values]
    else:
        adjusted_fitness = fitness_values
    
    total_fitness = sum(adjusted_fitness)
    
    if total_fitness == 0:
        return random.choice(flowers)
    
    pick = random.uniform(0, total_fitness)
    current = 0
    
    for i, flower in enumerate(flowers):
        current += adjusted_fitness[i]
        if current >= pick:
            return flower
    
    return flowers[-1]

def crossover_flowers(parent1, parent2):
    if random.random() > 0.65:
        return parent1, parent2
    
    offspring1 = Flower()
    offspring2 = Flower()
    
    crossover = random.randint(1, len(parent1.genes) - 1)
    
    offspring1.genes = parent1.genes[:crossover] + parent2.genes[crossover:]
    offspring2.genes = parent2.genes[:crossover] + parent1.genes[crossover:]
    
    offspring1.fitness = 0
    offspring1.hover_time = 0
    offspring2.fitness = 0
    offspring2.hover_time = 0
    
    return offspring1, offspring2

def mutate(flower):
    for i in range(len(flower.genes)):
        if random.random() < 0.05:
            if i == 0:
                flower.genes[i] = random.randint(1, 10)
            elif i == 10:
                flower.genes[i] = random.randint(0, 7)
            else:
                flower.genes[i] = random.randint(0, 255)
    return flower

def evolve_new_generation():
    global flowers, hover_start_times
    
    print("=== EVOLUTION IN PROGRESS ===")
    for i, flower in enumerate(flowers):
        print(f"Flower {i+1}: Fitness {flower.fitness:.2f}")
    
    new_population = []
    
    while len(new_population) < 8:
        parent1 = weighted_roulette_selection(flowers)
        parent2 = weighted_roulette_selection(flowers)
        
        offspring1, offspring2 = crossover_flowers(parent1, parent2)
        
        offspring1 = mutate(offspring1)
        offspring2 = mutate(offspring2)
        
        new_population.append(offspring1)
        if len(new_population) < 8:
            new_population.append(offspring2)
    
    for flower in new_population:
        flower.fitness = 0
        flower.hover_time = 0
    
    flowers = new_population[:8]
    hover_start_times = [None] * len(flowers)
    
    print("=== NEW GENERATION CREATED ===")
    print("All fitness values reset to 0 for new generation")
    return flowers

print("Evolution functions loaded! Now modify your pygame loop to add the button.")

Evolution functions loaded! Now modify your pygame loop to add the button.


In [14]:

pygame.init()
pygame.font.init()

WIDTH, HEIGHT = 800, 600
FLOWER_SPACING = WIDTH // 8
FPS = 60 
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flower Evolution - Hover to Increase Fitness")
clock = pygame.time.Clock()

hover_start_times = [None] * len(flowers)
FITNESS_RATE = 1.0

button_rect = pygame.Rect(WIDTH - 200, 10, 180, 40)
button_color = (100, 200, 100)
button_hover_color = (120, 220, 120)
button_text_color = (255, 255, 255)

generation_counter = 1

running = True
while running:
    current_time = time.time()
    mouse_pos = pygame.mouse.get_pos()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if button_rect.collidepoint(mouse_pos):
                total_fitness = sum(flower.fitness for flower in flowers)
                if total_fitness > 0:
                    evolve_new_generation()
                    generation_counter += 1
                else:
                    print("No flowers have fitness yet! Hover over flowers first.")

    screen.fill((255, 255, 255))

    for i, flower in enumerate(flowers):
        x_position = (i * FLOWER_SPACING) + (FLOWER_SPACING // 2)
        hover_area, is_hovering = draw_with_hover_tracking(screen, flower, x_position, mouse_pos)
        
        if is_hovering:
            if hover_start_times[i] is None:
                hover_start_times[i] = current_time
        else:
            if hover_start_times[i] is not None:
                session_duration = current_time - hover_start_times[i]
                flower.hover_time += session_duration
                flower.fitness = flower.hover_time * FITNESS_RATE
                hover_start_times[i] = None
    
    for i, flower in enumerate(flowers):
        if hover_start_times[i] is not None:
            current_session_time = current_time - hover_start_times[i]
            flower.fitness = (flower.hover_time + current_session_time) * FITNESS_RATE
    
    button_is_hovered = button_rect.collidepoint(mouse_pos)
    current_button_color = button_hover_color if button_is_hovered else button_color
    pygame.draw.rect(screen, current_button_color, button_rect)
    pygame.draw.rect(screen, (0, 0, 0), button_rect, 2)
    
    font = pygame.font.Font(None, 24)
    button_text = font.render("Evolve New Generation", True, button_text_color)
    text_rect = button_text.get_rect(center=button_rect.center)
    screen.blit(button_text, text_rect)
    
    font = pygame.font.Font(None, 36)
    instruction_text = font.render("Hover over flowers to increase their fitness!", True, (0, 0, 0))
    screen.blit(instruction_text, (WIDTH // 2 - 250, 50))
    
    gen_text = font.render(f"Generation: {generation_counter}", True, (0, 0, 0))
    screen.blit(gen_text, (10, 10))
    
    best_flower = max(flowers, key=lambda f: f.fitness)
    best_text = font.render(f"Best Fitness: {best_flower.fitness:.1f}", True, (0, 100, 0))
    screen.blit(best_text, (WIDTH // 2 - 100, HEIGHT - 50))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()


=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 2.03
Flower 2: Fitness 0.00
Flower 3: Fitness 0.00
Flower 4: Fitness 1.58
Flower 5: Fitness 0.03
Flower 6: Fitness 2.02
Flower 7: Fitness 1.78
Flower 8: Fitness 0.00
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 0.00
Flower 2: Fitness 0.00
Flower 3: Fitness 0.00
Flower 4: Fitness 1.18
Flower 5: Fitness 0.00
Flower 6: Fitness 0.00
Flower 7: Fitness 0.00
Flower 8: Fitness 0.00
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
=== EVOLUTION IN PROGRESS ===
Flower 1: Fitness 0.00
Flower 2: Fitness 0.00
Flower 3: Fitness 0.00
Flower 4: Fitness 0.00
Flower 5: Fitness 1.28
Flower 6: Fitness 0.77
Flower 7: Fitness 0.99
Flower 8: Fitness 0.00
=== NEW GENERATION CREATED ===
All fitness values reset to 0 for new generation
